# Ejercicio 1. Red Neuronal en Keras y ScikitLearn

**Autor:** Ángel Manuel Calzado Llamas  
**UVUS:** angcallla

## Importaciones necesarias

In [1]:
# Importamos los paquetes necesarios
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation

## 1. Dataset: Descripción y carga
### Información general

El objetivo de este ejercicio es la creación de redes neurales artificiales que se ajusten a los datos que se nos proporcionan. 

**Título del Dataset**: Pima Indians Diabetes Database  
**Descripción**: Diagnóstico de diabetes en Indios Pima, en base a sus datos personales (edad, núm. de embarazos, ...)  
**Fuentes**:
- (a) Original owners: National Institute of Diabetes and Digestive and
                        Kidney Diseases
- (b) Donor of database: Vincent Sigillito (vgs@aplcen.apl.jhu.edu)
                          Research Center, RMI Group Leader
                          Applied Physics Laboratory
                          The Johns Hopkins University
                          Johns Hopkins Road
                          Laurel, MD 20707
                          (301) 953-6231
- (c) Date received: 9 May 1990  

**Objetivo**: Creación de redes neurales que decidan si un Indio Pima es diabético o no

### Estructura de los datos

Vamos a explicar ahora el tipo de características que contiene cada instancia, es decir, cada persona. Lo primero que debemos saber es que, al momento de trabajar con estos datos, ya se ha realizado un tratamiento de datos ausentes y se han **normalizado**, lo cual es importante de cara al entrenamiento de la red para que no de más importancia a unas características que a otras.

No obstante, se nos comenta que en el dataset existen algunos valores **muy cercanos a 0**, que podrían considerarse como **valores ausentes**. La documentación explica que se han tratado como si fueran valores reales, que añaden algo de **error o ruido** a los datos. Nosotros los dejaremos incluídos para estudiar como trabajan nuestros modelos que estas casuísticas.

Las características de las instancias (previamente normalizadas) son:

0. `Número de embarazos`: continuo (con 34 casos mayores a 10)
1. `Concentración de glucosa`: continuo (5 casos de valor 0, indicando valor ausente o erróneo, lo dejamos como ya dijimos)
2. `Presión sanguínea diástole (mm Hg)`: continuo (35 casos con valor 0, valor posterior más alto de 24)
3. `Grosor del pliegue cutáneo del tríceps (mm)`: continuo (227 casos con valor 0, valor posterior más alto de 7)
4. `Insulina sérica de 2 horas (mu U/ml)`: continuo (347 casos con valor 0, valor posterior más alto de 14)
5. `Índice de masa corporal (peso in kg/(altura in m)^2)`: continuo (11 casos con valor 0, valor posterior más alto de 18)
6. `Función de pedigrí de la diabetes`: continuo
7. `Edad (años)`: continuo
8. `Variable de clasificación`: En dos columnas *diabetes* y *no_diabetes*, siendo 1 en aquella columna en la que se corresponda la instancia

Distribución de las clases:  
   0:     65%    500  no diabetes  
   1:     35%    268  diabetes  
total:         768  

### Carga de los datos

Los datos se encuentran en el dichero adjunto *diabetes.csv*. Para leer el fichero, utilizaremos la libería de `pandas`, concretamente su función `read_csv` indicándole la ruta del fichero.

In [2]:
# Lectura del dataset
data = pd.read_csv("diabetes.csv")

A continuación, dividimos las columnas de nuestros datos entre características y valor de clasificación, creando un array 2D `X` que guarda las características 0-8 y un array 1D con el valor de clasificación de cada instancia (siendo 1 diabetes y 0 no diabetes).

Para ello utilizamos la librería `numpy` especificando la creación de dos nuevos arrays con `array` indicando los dos nuevos arrays que queremos crear. Utilizamos la función `iloc` para especificar las columnas que queremos seleccionar en `X` e `Y`.

In [3]:
# División entre características y valor de clasificación
X, Y = np.array(data.iloc[:,0:8]), np.array(data.iloc[:,9])

Como recordará, explicamos que el valor de clasificación realmente se componía de dos columnas: *diabetes* y *no_diabetes*. Con motivo de no almacenar características redundantes, sabemos que si en la columna *diabetes* una instancia es 1, significa diabetes y si es 0, significa que en la columna *no_diabetes* es 1, y que por tanto significa no diabetes.

Vamos a mostrar la forma de nuestros Arrays para comprobar que hemos hecho bien los tratamientos hasta ahora. Utilizamos la función `shape` para conocer la "forma" de `X` e `Y`

In [4]:
# Forma de X e Y
print("Forma X: ", X.shape)
print("Forma Y: ", Y.shape)

Forma X:  (768, 8)
Forma Y:  (768,)


Efectivamente, hemos almacenado en `X`las 8 características de los datos y en `Y` el valor de clasificación. Mostramos un ejemplo de ambos antes de pasar al siguiente apartado

In [5]:
print(X[0:10])

[[0.176471  0.605     0.42623   0.        0.        0.536513  0.0209223
  0.0666667]
 [0.352941  0.72      0.590164  0.27      0.269504  0.505216  0.0755764
  0.316667 ]
 [0.117647  0.875     0.721311  0.        0.        0.341282  0.105892
  0.0166667]
 [0.705882  0.605     0.639344  0.17      0.        0.394933  0.0772844
  0.683333 ]
 [0.117647  0.535     0.606557  0.3       0.118203  0.500745  0.139197
  0.0333333]
 [0.882353  0.68      0.57377   0.32      0.130024  0.552906  0.0320239
  0.366667 ]
 [0.470588  0.545     0.622951  0.39      0.134752  0.415797  0.239966
  0.166667 ]
 [0.117647  0.405     0.491803  0.22      0.        0.412817  0.0905209
  0.0666667]
 [0.235294  0.985     0.57377   0.39      0.879433  0.546945  0.961144
  0.166667 ]
 [0.        0.525     0.737705  0.        0.        0.441133  0.0508113
  0.416667 ]]


In [6]:
print(Y[0:10])

[0 1 1 1 1 0 0 1 1 1]


## 2. Preparación de los datos para ser usados en tensorflow

Como explicamos en el apartado anterior, los atributos de los datos son todos continuos y se encuentran normalizados, por lo que es un preprocesamiento que nos ahorramos. Por otro lado, podemos afirmar que todos ellos son **informativos, discriminativos y no redundantes** por lo que no sería necesario eliminar ninguna columna.

No obstante, existen algunos valores que son 0 en algunos atributos, pero los tenemos controlados, sabiendo cuantos de ellos son para cada columna. Merece la pena dejarlos, para observar lo bien que puede funcionar nuestro modelo con la existencia de ruido.

Dicho esto, no sería necesario ningún tipo de preprocesamiento sobre los atributos. Pasamos entonces a dividir el conjunto total de datos entre **entrenamiento y prueba**. No vamos a considerar un conjunto de validación porque pretendemos crear distintos modelos y evaluarlos por separado, es decir, no estamos en busca de un único modelo del que queramos optimizar sus hiperparámetros.

Las proporciones escogidas para la división de los datos será de **80%-20%** debido a que apenas hay instancias en los datos y necesitamos un número razonable de estos para que el modelo pueda generalizar bien el problema. Además, esta división será **estratificada**, es decir, que existirá el mismo porcentaje de diabéticos que de no diabéticos tanto en el conjunto de entrenamiento como en el de test. Esto ayudará a evitar aprendizajes no acordes a la realidad de los datos.

Dividimos, entonces, los datos en entrenamiento y prueba. Utilizamos el parámetro `stratify` de la función para estratificar los datos en base a los valores de `Y`:

In [7]:
# Dividimos los datos en entrenamiento y prueba
train_X, test_X, train_y, test_y = train_test_split(X, Y, train_size=0.8, test_size=0.2, stratify=Y, random_state=0)

Confirmemos si las proporciones son semejantes

In [8]:
print("Proporción de diabéticos en entrenamiento:", 
      train_y[train_y==1].shape[0]/train_y.shape[0])
print("Proporción de diabéticos en prueba:", 
      test_y[test_y==1].shape[0]/test_y.shape[0])

Proporción de diabéticos en entrenamiento: 0.6514657980456026
Proporción de diabéticos en prueba: 0.6493506493506493


Lo son.



## 3. Modelos creados

En los siguientes subapartados definimos cada uno de los modelos que van a ser implementados, así como su implementación. Indicaremos para cada modelo, cuales son sus **aspectos**.

Algo que vamos a ir adelantando ya y que será **igual** en todos los modelos es el número de nodos de la **capa de salida** y su función de activación. Al ser este un problema de **clasificación binaria** (1 - diabetes, 0 - no diabetes), la capa de salida tendrá **un solo nodo** con función de activación **sigmoide**, cuya salida se interpretará como una **probabilidad** de pertenecer a la clase positiva (diabetes).

### 3.1 Primer modelo

Los aspectos del modelo son:
- `1 capa oculta con 16 nodos`: Similar a la estructura de la práctica 3.2, una capa oculta evitará el sobreajuste al no tener tantos pesos y 16 nodos es un buen valor para la GPU al ser potencia de 2.
- `función de activación ReLU`: Al ser el más popular para las capas ocultas y aportar mejor rendimiento en la búsqueda de la convergencia.
- `factor de aprendizaje 0.01, 10-50-100 epochs, función de coste de perceptrón`: El factor de aprendizaje ha de ser bajo para evitar las *Dying ReLUs* y usaremos distintos epochs para responder a las preguntas que se nos piden.  

    Utilizamos la función de coste del perceptrón al ser este un problema de clasificación binaria. Recordemos que utilizamos aquella función que varía según sea el valor "y" esperado 0 o 1, para que la función en cada caso alcance el mínimo en el valor esperado y tienda a infinito cuando más se aleje de él.
- `Método de optimización Adam`: El cual es el más recomendado para empezar.

![diagrama_modelo_1](imgej1/modelo1.png)

Lo implementamos. Comenzamos inicializando el modelo secuencial:

In [9]:
model1_n16_relu = Sequential()

Definimos las 8 variables de entrada y los 16 nodos de la capa oculta con función de activación ReLU:

In [10]:
model1_n16_relu.add(Dense(16, input_shape=(8,)))
model1_n16_relu.add(Activation('relu'))

la capa de salida con un solo nodo con función de activación sigmoide

In [11]:
model1_n16_relu.add(Dense(1))
model1_n16_relu.add(Activation('sigmoid'))

Y finalizamos con la definición del optimizador, función de pérdida y evaluador que emplearemos el cual será la medida de precisión del modelo, que calcula el porcentaje de instancia predichas correctamente frente el total

In [12]:
model1_n16_relu.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=["accuracy"])

En definitiva, el modelo es el siguiente:

In [13]:
model1_n16_relu.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                144       
_________________________________________________________________
activation (Activation)      (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 161
Trainable params: 161
Non-trainable params: 0
_________________________________________________________________


### 3.2 Segundo modelo

Los aspectos del modelo son:
- `2 capa ocultas con 8 nodos`: La adición de más capas implicará mayor ajuste sobre los datos, por si el modelo anterior pudiera quedar infrajustado.
- `función de activación Leaky ReLU`: En todas las capas ocultas, con motivo de evitar las *dying ReLU*
- `factor de aprendizaje 0.1, 10-50-100 epochs, función de coste de perceptrón`: El factor de aprendizaje es más alto porque ya no habría necesidad de evitar *Dying ReLU* por la naturaleza de la propia función y usaremos distintos epochs para responder a las preguntas que se nos piden.  

    Utilizamos la función de coste del perceptrón al ser este un problema de clasificación binaria. Como la naturaleza del problema es la misma, independientemente del modelo, dicha función no varía
    
- `Método de optimización RMSProp`: Para compararlo con Adam ya que a priori sabemos que aportan resultados similares.

![diagrama_modelo_2](imgej1/modelo2.png)

Lo implementamos de la misma forma que el modelo 1

In [14]:
# instanciamos el modelo
model2_n8_leakyRelu = Sequential()

In [15]:
# añadimos las 2 capas ocultas con 8 nodos cada una y con función de activación Leaky ReLU
model2_n8_leakyRelu.add(Dense(8, input_shape=(8,)))
model2_n8_leakyRelu.add(Activation(tf.keras.layers.LeakyReLU()))

model2_n8_leakyRelu.add(Dense(8, input_shape=(8,)))
model2_n8_leakyRelu.add(Activation(tf.keras.layers.LeakyReLU()))

In [16]:
# añadimos la capa de salida con función de activación sigmoide
model2_n8_leakyRelu.add(Dense(1))
model2_n8_leakyRelu.add(Activation('sigmoid'))

In [17]:
# compilamos indicando el optimizador RMSprop, función de pérdida del perceptrón y accuracy como método de evaluación
model2_n8_leakyRelu.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.01), loss='binary_crossentropy', metrics=["accuracy"])

In [18]:
# Mostramos el resumen del modelo
model2_n8_leakyRelu.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 8)                 72        
_________________________________________________________________
activation_2 (Activation)    (None, 8)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 72        
_________________________________________________________________
activation_3 (Activation)    (None, 8)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 9         
_________________________________________________________________
activation_4 (Activation)    (None, 1)                 0         
Total params: 153
Trainable params: 153
Non-trainable params: 0
________________________________________________________

### 3.3 Tercer modelo

Los aspectos del modelo son:
- `3 capa ocultas con 16, 8 y 4 nodos`: Se tratará de una estructura de red en forma de "embudo" de forma que al inicio calculará más características y posteriormente menos.
- `función de activación ELU, LReLU y ReLU`: Respectivamente, en cada una de las capas, para ir restringiendo más los valores negativos.
- `factor de aprendizaje 1, 10-50-100 epochs, función de coste de perceptrón`: Factor de aprendizaje máximo. Se sabe que cuanto mayor es el ratio de aprendizaje, menos epochs son necesarios.

    Utilizamos la función de coste del perceptrón al ser este un problema de clasificación binaria. Como la naturaleza del problema es la misma, independientemente del modelo, dicha función no varía
    
- `Método de optimización Adadelta`: Último de los más recomendados, con motivo de compararlo con los anteriores.

![diagrmaa_modelo_3](imgej1/modelo3.png)

Lo implementamos de la misma forma que con el modelo 2 y 3

In [19]:
# instanciamos el modelo
model3_embudo = Sequential()

In [20]:
# añadimos las 3 capas ocultas con 16,8,4 nodos y con función de activación ELU, Leaky ReLU y ReLU; respectivamente
model3_embudo.add(Dense(16, input_shape=(8,)))
model3_embudo.add(Activation('elu'))

model3_embudo.add(Dense(8, input_shape=(16,)))
model3_embudo.add(Activation(tf.keras.layers.LeakyReLU()))

model3_embudo.add(Dense(4, input_shape=(8,)))
model3_embudo.add(Activation("relu"))

In [21]:
# añadimos la capa de salida con función de activación sigmoide
model3_embudo.add(Dense(1))
model3_embudo.add(Activation('sigmoid'))

In [22]:
# compilamos indicando el optimizador RMSprop, función de pérdida del perceptrón y accuracy como método de evaluación
model3_embudo.compile(optimizer=keras.optimizers.Adadelta(learning_rate=1), loss='binary_crossentropy', metrics=["accuracy"])

## 4. Entrenamiento y evaluación

Una vez creados y estructurados los modelos, pasamos a entrenarlos y evaluarlos. Para entrenarlos hacemos uso de la función `fit` pasándole el conjunto de entrenamiento. Por otro lado, para evaluarlo, utilizamos la función `evaluate` tanto para el conjunto de entrenamiento como el de prueba; necesario para conocer si el modelo generaliza adecuadamente el problema.

Acerca del número de epochs y el número de batches, elegiremos unas opciones u otras dependiendo del modelo.

### 4.1 Modelo 1

Al ser este un modelo no muy complejo y no hay muchos datos, haremos un primer entrenamiento con 100 epochs y tamaño de batch 1.

In [23]:
# (1) Entrenamos sobre el conjunto de entrenamiento
model1_n16_relu.fit(train_X,train_y,epochs=100,batch_size=1)

Epoch 1/100
614/614 [==============================] - 2s 2ms/step - loss: 0.6333 - accuracy: 0.6450: 0s - loss: 0.6681 -  - ETA: 0s - loss: 0.6375 - accuracy
Epoch 2/100
614/614 [==============================] - 1s 2ms/step - loss: 0.5670 - accuracy: 0.7003: 
Epoch 3/100
614/614 [==============================] - 1s 2ms/step - loss: 0.5372 - accuracy: 0.7362
Epoch 4/100
614/614 [==============================] - 1s 2ms/step - loss: 0.5118 - accuracy: 0.7443
Epoch 5/100
614/614 [==============================] - 1s 2ms/step - loss: 0.5012 - accuracy: 0.7459
Epoch 6/100
614/614 [==============================] - 1s 2ms/step - loss: 0.4873 - accuracy: 0.7704
Epoch 7/100
614/614 [==============================] - 1s 2ms/step - loss: 0.4982 - accuracy: 0.7606
Epoch 8/100
614/614 [==============================] - 1s 2ms/step - loss: 0.4853 - accuracy: 0.7443
Epoch 9/100
614/614 [==============================] - 1s 2ms/step - loss: 0.4882 - accuracy: 0.7476
Epoch 10/100
614/614 [=========

614/614 [==============================] - 1s 1ms/step - loss: 0.4226 - accuracy: 0.7997
Epoch 80/100
614/614 [==============================] - 1s 1ms/step - loss: 0.4219 - accuracy: 0.7899
Epoch 81/100
614/614 [==============================] - 1s 2ms/step - loss: 0.4199 - accuracy: 0.7752
Epoch 82/100
614/614 [==============================] - 1s 1ms/step - loss: 0.4159 - accuracy: 0.7801
Epoch 83/100
614/614 [==============================] - 1s 2ms/step - loss: 0.4182 - accuracy: 0.7932
Epoch 84/100
614/614 [==============================] - 1s 2ms/step - loss: 0.4288 - accuracy: 0.7932
Epoch 85/100
614/614 [==============================] - 1s 2ms/step - loss: 0.4205 - accuracy: 0.7997
Epoch 86/100
614/614 [==============================] - 1s 2ms/step - loss: 0.4168 - accuracy: 0.7997
Epoch 87/100
614/614 [==============================] - 1s 2ms/step - loss: 0.4233 - accuracy: 0.7850
Epoch 88/100
614/614 [==============================] - 1s 2ms/step - loss: 0.4187 - accuracy: 

Pasamos a evaluar el modelo

In [24]:
# (1) Evaluamos sobre el conjunto de entrenamiento y test
loss, accuracy = model1_n16_relu.evaluate(train_X, train_y, verbose=0)
print("Coste = {:.2f},Accuracy = {:.2f}".format(loss,accuracy))
loss, accuracy = model1_n16_relu.evaluate(test_X, test_y, verbose=0)
print("Coste = {:.2f},Accuracy = {:.2f}".format(loss,accuracy))

Coste = 0.40,Accuracy = 0.80
Coste = 0.53,Accuracy = 0.75


Dejaremos los comentarios acerca del proceso de entrenamiento para el apartado de "análisis de los resultados".

Probamos también con 10 epochs y el mismo tamaño de batch para comparar

In [25]:
# (2) Entrenamos sobre el conjunto de entrenamiento
model1_n16_relu.fit(train_X,train_y,epochs=10,batch_size=1)

Epoch 1/10
614/614 [==============================] - 1s 2ms/step - loss: 0.4096 - accuracy: 0.7948
Epoch 2/10
614/614 [==============================] - 1s 2ms/step - loss: 0.4162 - accuracy: 0.7883
Epoch 3/10
614/614 [==============================] - 1s 2ms/step - loss: 0.4101 - accuracy: 0.7899
Epoch 4/10
614/614 [==============================] - 1s 2ms/step - loss: 0.4196 - accuracy: 0.7801
Epoch 5/10
614/614 [==============================] - 1s 2ms/step - loss: 0.4140 - accuracy: 0.7932
Epoch 6/10
614/614 [==============================] - 1s 2ms/step - loss: 0.4160 - accuracy: 0.7883
Epoch 7/10
614/614 [==============================] - 1s 2ms/step - loss: 0.4190 - accuracy: 0.7866
Epoch 8/10
614/614 [==============================] - 1s 2ms/step - loss: 0.4181 - accuracy: 0.7899
Epoch 9/10
614/614 [==============================] - 1s 2ms/step - loss: 0.4109 - accuracy: 0.7818
Epoch 10/10
614/614 [==============================] - 1s 2ms/step - loss: 0.4147 - accuracy: 0.7866

In [26]:
# (2) Evaluamos el modelo
loss, accuracy = model1_n16_relu.evaluate(train_X, train_y, verbose=0)
print("Coste = {:.2f},Accuracy = {:.2f}".format(loss,accuracy))
loss, accuracy = model1_n16_relu.evaluate(test_X, test_y, verbose=0)
print("Coste = {:.2f},Accuracy = {:.2f}".format(loss,accuracy))

Coste = 0.39,Accuracy = 0.81
Coste = 0.55,Accuracy = 0.73


### 4.2 Modelo 2
Para este modelo usaremos 10 y 100 epochs, y también variaremos el tamaño de batch siendo mayor cuando más epochs haya.

In [35]:
# (1) Entrenamos sobre el conjunto de entrenamiento
model2_n8_leakyRelu.fit(train_X,train_y,epochs=100,batch_size=32)

Epoch 1/100
20/20 [==============================] - 0s 2ms/step - loss: 0.5032 - accuracy: 0.7622
Epoch 2/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4504 - accuracy: 0.7915
Epoch 3/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4606 - accuracy: 0.7834
Epoch 4/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4578 - accuracy: 0.7752
Epoch 5/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4606 - accuracy: 0.7850
Epoch 6/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4496 - accuracy: 0.7818
Epoch 7/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4568 - accuracy: 0.7704
Epoch 8/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4496 - accuracy: 0.7818
Epoch 9/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4480 - accuracy: 0.7883
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4490 - accuracy: 0.7883
Epoch 11/

20/20 [==============================] - 0s 3ms/step - loss: 0.4253 - accuracy: 0.7932
Epoch 83/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4292 - accuracy: 0.7932
Epoch 84/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4344 - accuracy: 0.7834
Epoch 85/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4281 - accuracy: 0.7785
Epoch 86/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4289 - accuracy: 0.7801
Epoch 87/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4252 - accuracy: 0.7915
Epoch 88/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4312 - accuracy: 0.7883
Epoch 89/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4245 - accuracy: 0.7915
Epoch 90/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4275 - accuracy: 0.8029
Epoch 91/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4263 - accuracy: 0.7866
Epoch 92/100


In [36]:
# (1) Evaluamos sobre el conjunto de entrenamiento y test
loss, accuracy = model2_n8_leakyRelu.evaluate(train_X, train_y, verbose=0)
print("Coste = {:.2f},Accuracy = {:.2f}".format(loss,accuracy))
loss, accuracy = model2_n8_leakyRelu.evaluate(test_X, test_y, verbose=0)
print("Coste = {:.2f},Accuracy = {:.2f}".format(loss,accuracy))

Coste = 0.41,Accuracy = 0.80
Coste = 0.47,Accuracy = 0.77


In [29]:
# (2) Entrenamos sobre el conjunto de entrenamiento
model2_n8_leakyRelu.fit(train_X,train_y,epochs=10,batch_size=1)

Epoch 1/10
614/614 [==============================] - 2s 3ms/step - loss: 0.5344 - accuracy: 0.7443
Epoch 2/10
614/614 [==============================] - 2s 3ms/step - loss: 0.5094 - accuracy: 0.7573
Epoch 3/10
614/614 [==============================] - 2s 3ms/step - loss: 0.5176 - accuracy: 0.7606
Epoch 4/10
614/614 [==============================] - 2s 3ms/step - loss: 0.5244 - accuracy: 0.7687
Epoch 5/10
614/614 [==============================] - 2s 3ms/step - loss: 0.5244 - accuracy: 0.7492
Epoch 6/10
614/614 [==============================] - 2s 3ms/step - loss: 0.5166 - accuracy: 0.7557
Epoch 7/10
614/614 [==============================] - 2s 3ms/step - loss: 0.5097 - accuracy: 0.7573
Epoch 8/10
614/614 [==============================] - 2s 3ms/step - loss: 0.5173 - accuracy: 0.7508
Epoch 9/10
614/614 [==============================] - 2s 3ms/step - loss: 0.5127 - accuracy: 0.7345
Epoch 10/10
614/614 [==============================] - 2s 3ms/step - loss: 0.5016 - accuracy: 0.7671

In [30]:
# (2) Evaluamos sobre el conjunto de entrenamiento y test
loss, accuracy = model2_n8_leakyRelu.evaluate(train_X, train_y, verbose=0)
print("Coste = {:.2f},Accuracy = {:.2f}".format(loss,accuracy))
loss, accuracy = model2_n8_leakyRelu.evaluate(test_X, test_y, verbose=0)
print("Coste = {:.2f},Accuracy = {:.2f}".format(loss,accuracy))

Coste = 0.63,Accuracy = 0.72
Coste = 0.62,Accuracy = 0.75


### 4.3 Modelo 3
Para este modelo usaremos 50 y 100 epochs, y también variaremos el tamaño de batch siendo mayor cuando más epochs haya.

In [31]:
# (1) Entrenamos sobre el conjunto de entrenamiento
model3_embudo.fit(train_X,train_y,epochs=50,batch_size=32)

Epoch 1/50
20/20 [==============================] - 1s 2ms/step - loss: 0.6763 - accuracy: 0.6564
Epoch 2/50
20/20 [==============================] - 0s 2ms/step - loss: 0.6612 - accuracy: 0.6515
Epoch 3/50
20/20 [==============================] - 0s 2ms/step - loss: 0.6544 - accuracy: 0.6515
Epoch 4/50
20/20 [==============================] - 0s 3ms/step - loss: 0.6457 - accuracy: 0.6515
Epoch 5/50
20/20 [==============================] - 0s 3ms/step - loss: 0.6369 - accuracy: 0.6515
Epoch 6/50
20/20 [==============================] - 0s 3ms/step - loss: 0.6272 - accuracy: 0.6515
Epoch 7/50
20/20 [==============================] - 0s 3ms/step - loss: 0.6173 - accuracy: 0.6515
Epoch 8/50
20/20 [==============================] - 0s 2ms/step - loss: 0.6079 - accuracy: 0.6515
Epoch 9/50
20/20 [==============================] - 0s 2ms/step - loss: 0.5979 - accuracy: 0.6515
Epoch 10/50
20/20 [==============================] - 0s 2ms/step - loss: 0.5900 - accuracy: 0.6515
Epoch 11/50
20/20 [

In [32]:
# (1) Evaluamos sobre el conjunto de entrenamiento y test
loss, accuracy = model3_embudo.evaluate(train_X, train_y, verbose=0)
print("Coste = {:.2f},Accuracy = {:.2f}".format(loss,accuracy))
loss, accuracy = model3_embudo.evaluate(test_X, test_y, verbose=0)
print("Coste = {:.2f},Accuracy = {:.2f}".format(loss,accuracy))

Coste = 0.50,Accuracy = 0.75
Coste = 0.51,Accuracy = 0.75


In [33]:
# (2) Entrenamos sobre el conjunto de entrenamiento
model3_embudo.fit(train_X,train_y,epochs=100,batch_size=50)

Epoch 1/100
13/13 [==============================] - 0s 3ms/step - loss: 0.4955 - accuracy: 0.7492
Epoch 2/100
13/13 [==============================] - 0s 3ms/step - loss: 0.4964 - accuracy: 0.7427
Epoch 3/100
13/13 [==============================] - 0s 3ms/step - loss: 0.4971 - accuracy: 0.7557
Epoch 4/100
13/13 [==============================] - 0s 3ms/step - loss: 0.4962 - accuracy: 0.7443
Epoch 5/100
13/13 [==============================] - 0s 3ms/step - loss: 0.4946 - accuracy: 0.7508
Epoch 6/100
13/13 [==============================] - 0s 3ms/step - loss: 0.4939 - accuracy: 0.7557
Epoch 7/100
13/13 [==============================] - 0s 3ms/step - loss: 0.4944 - accuracy: 0.7573
Epoch 8/100
13/13 [==============================] - 0s 3ms/step - loss: 0.4938 - accuracy: 0.7476
Epoch 9/100
13/13 [==============================] - 0s 3ms/step - loss: 0.4937 - accuracy: 0.7492
Epoch 10/100
13/13 [==============================] - 0s 2ms/step - loss: 0.4931 - accuracy: 0.7508
Epoch 11/

13/13 [==============================] - 0s 5ms/step - loss: 0.4819 - accuracy: 0.7590
Epoch 83/100
13/13 [==============================] - 0s 2ms/step - loss: 0.4850 - accuracy: 0.7459
Epoch 84/100
13/13 [==============================] - 0s 2ms/step - loss: 0.4832 - accuracy: 0.7476
Epoch 85/100
13/13 [==============================] - 0s 3ms/step - loss: 0.4863 - accuracy: 0.7541
Epoch 86/100
13/13 [==============================] - ETA: 0s - loss: 0.5539 - accuracy: 0.72 - 0s 2ms/step - loss: 0.4862 - accuracy: 0.7427
Epoch 87/100
13/13 [==============================] - 0s 3ms/step - loss: 0.4856 - accuracy: 0.7541
Epoch 88/100
13/13 [==============================] - 0s 3ms/step - loss: 0.4830 - accuracy: 0.7476
Epoch 89/100
13/13 [==============================] - 0s 2ms/step - loss: 0.4836 - accuracy: 0.7443
Epoch 90/100
13/13 [==============================] - 0s 2ms/step - loss: 0.4830 - accuracy: 0.7557
Epoch 91/100
13/13 [==============================] - 0s 2ms/step - los

In [34]:
# (2) Evaluamos sobre el conjunto de entrenamiento y test
loss, accuracy = model3_embudo.evaluate(train_X, train_y, verbose=0)
print("Coste = {:.2f},Accuracy = {:.2f}".format(loss,accuracy))
loss, accuracy = model3_embudo.evaluate(test_X, test_y, verbose=0)
print("Coste = {:.2f},Accuracy = {:.2f}".format(loss,accuracy))

Coste = 0.48,Accuracy = 0.75
Coste = 0.50,Accuracy = 0.79


## 5. Análisis de los resultados

### 5.1 ¿Qué podemos decir del proceso de entrenamiento?

Durante el entrenamiento, se nos van presentando los valores de la función de pérdida y precisión en cada epoch. Esta información ha sido útil para ir viendo como durante la búsqueda de mínimo local o global, ha ido variando los modelos.

Prácticamente todos los modelos **han actuado de la misma manera**, encontrando desde el inicio un mínimo local que aportaba una precisión de, al menos, más del 75%, y a medida que iban pasando epochs, iba aumentando o disminuyendo **sin bajar del 70%**. Esto es particularmente importante porque ha sido posible obtener buenos resultados con pocos epochs, lo que mejora la **computación** del modelo.

### 5.2 ¿Qué podemos decir del proceso de evaluación?

En cuanto a la evaluación sobre el conjunto de entrenamiento y test, los resultados **han sido semejantes** en todos los modelos con apróximadamente un valor de **80%** en ambos casos. Un valor del 80% sobre el conjunto de entrenamiento no es un resultado que podamos considerar del todo bueno, dando a entender que es probable que el modelo se encuentre **infrajustado**. No obstante, ha podido ver como hemos probado distintos modelos, uno más complejo que el anterior y los resultados han sido los mismos. Por tanto, podríamos pensar en que o necesitamos más características o la naturaleza del problema es confusa y no hemos logrado la configuración adecuada.

### 5.3 ¿Qué modelo es mejor?

El mejor modelo es aquel que posee un mejor balance entre velocidad de procesado y calidad de los resultados. Dado que con nuestra configuración los resultados han sido similares, pasamos a hablar de velocidad, donde podríamos decir que el **primer modelo**, al ser **más simple** y, por tanto, computacionalmente **más rápido**, se podría considerar como mejor para el problema al que nos estamos enfrentando.

## 6. Bibliografía

- Módulos 1, 2 y 3 de la asignatura Deep Learning coordinada por Miguel Ángel Martínez del Amor para el Máster Oficial de Ingeniería Informática en la Universidad de Sevilla.
- Módulos 6, 7 y 8 de la asigantura Aprendizaje Automático acerca de redes neuronales, preprocesamiento y evaluación de los datos, por Francisco J. Martín Mateos y José Luis Ruiz Reina para el Máster Oficial de Ingeniería Informática en la Universidad de Sevilla.
- Documentación keras: https://keras.io/api/
- Documentación numpy: https://numpy.org/doc/stable/reference/
- Documentación tensorflow: https://www.tensorflow.org/api_docs
